In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('unSuper').getOrCreate()

In [3]:
invoices = spark.read.csv(
    '/Users/mj/OneDrive - Aviana Global Technologies Inc/code/datasets/original/AvianaML_dbo_invoice.csv',
    header=True, inferSchema=True)

In [4]:
invoices.createOrReplaceTempView('invoices')

In [5]:
suppliers = spark.sql('''
select * from (
select ProjectID, 
`Vendor Name` as Supplier,
cast(year(`Invoice Date`) as INT) as Year,
round(avg(`Gross Invoice Amount`), 2) as Avg_Invoice_Amount,
count(*) as Total_Invoice_Count
from invoices
group by ProjectID, `Vendor Name`, year(`Invoice Date`)) t
where Total_Invoice_Count > 50
order by Avg_Invoice_Amount desc
''')
suppliers.toPandas().to_csv('datasets/TopSuppliers.csv', index=False)
suppliers.show()

+----------------+--------------------+----+------------------+-------------------+
|       ProjectID|            Supplier|Year|Avg_Invoice_Amount|Total_Invoice_Count|
+----------------+--------------------+----+------------------+-------------------+
|   Anadarko_2014|TRANSOCEAN DEEPWA...|2013|     1.892445989E8|                 56|
|   Anadarko_2014|BP AMERICA PRODUC...|2011|     7.934083274E7|                 58|
|   Anadarko_2014|           SONATRACH|2013|     5.552295498E7|                 65|
|   Anadarko_2014|DOLPHIN DRILLING LTD|2014|     4.632742989E7|                 82|
|        ETP_2017|ENERGY TRANSFER E...|2016|     4.600396043E7|                 57|
|   Anadarko_2018|           SONATRACH|2017|     4.584532274E7|                109|
|     Tesoro_2014|BP OIL SUPPLY COM...|2013|     4.535619872E7|                135|
|   Anadarko_2014| JPMORGAN CHASE BANK|2012|     4.310264617E7|                 59|
|     Tesoro_2014|PETROCHINA INTERN...|2013|     2.309316016E7|             

In [8]:
doctypes = spark.sql('''
select * from (
select ProjectID, 
`Vendor Name` as Supplier, 
`Document Type` as DocType, 
year(`Invoice Date`) as Year,
count(*) as Count,
round(avg(`Gross Invoice Amount`), 2) as Avg_Invoice_Amount
from invoices 
group by ProjectID, `Vendor Name`, `Document Type`, year(`Invoice Date`)) t
where Count > 50
order by Year desc
''')
doctypes.toPandas().to_csv('datasets/TopDocTypes.csv', index=False)
doctypes.show()

+------------+--------------------+-------+----+-----+------------------+
|   ProjectID|            Supplier|DocType|Year|Count|Avg_Invoice_Amount|
+------------+--------------------+-------+----+-----+------------------+
|Sanchez_2018|CORE-TECH WIRELIN...|  APINV|2018|  149|           1041.67|
|Sanchez_2018|RDZ OILFIELD LEAS...|  APINV|2018|  160|            960.57|
|Sanchez_2018| PLATINUM PTS II LLC|  APINV|2018|  193|           2571.32|
|Sanchez_2018|     RIG RUNNERS INC|  APINV|2018|  192|           3145.88|
|Sanchez_2018| BELL SUPPLY COMPANY|  APINV|2018|   87|          12635.16|
|Sanchez_2018|  DETECHTION USA INC|  APINV|2018|   60|           3394.25|
|Sanchez_2018|SPN WELL SERVICE ...|  APINV|2018|   85|           7670.76|
|Sanchez_2018|PICO PROPANE OPER...|  APINV|2018|  177|            2060.2|
|Sanchez_2018|BASIC ENERGY SERV...|  APINV|2018|  131|           3590.15|
|Sanchez_2018|NOV LP - COMPLETI...|  APINV|2018|   71|            6101.5|
|Sanchez_2018|BEAR CREEK SERVIC...|  A

In [9]:
projects = spark.sql('''
select * from (
select ProjectID, 
cast(year(`Invoice Date`) as INT) as Year, 
round(avg(`Gross Invoice Amount`), 2) as Avg_Invoice_Amount,
count(*) as Total_Invoice_Count
from invoices
group by ProjectID, Year) t
where Total_Invoice_Count > 100
order by Avg_Invoice_Amount desc
''')
projects.toPandas().to_csv('datasets/TopProjects.csv', index=False)
projects.show()

+--------------------+----+------------------+-------------------+
|           ProjectID|Year|Avg_Invoice_Amount|Total_Invoice_Count|
+--------------------+----+------------------+-------------------+
|       Anadarko_2014|2011|         126405.18|             985129|
|       Anadarko_2014|2013|           88269.3|            1158136|
|       Anadarko_2014|2012|          87770.75|            1054107|
|       Anadarko_2014|2014|          83507.56|             907977|
|            ETP_2017|2016|          83215.21|             343188|
|            ETP_2016|2015|           81827.3|             367371|
|       Anadarko_2018|2017|          80426.09|             688051|
|    NobleEnergy_2018|2017|          54034.81|             232092|
|    MarathonOil_2013|2013|          51347.04|             201778|
|            ETP_2016|2014|          47833.97|              32125|
|    NobleEnergy_2018|2016|          41923.81|             200571|
|         Tesoro_2014|2011|          41036.58|             527

In [10]:
projects.count()

48

In [11]:
suppliers.count()

26441

In [12]:
doctypes.count()

27541